In [1]:
from loader import load_reviews_to_list
from text_search import *
from spatial_search import *

In [2]:
path = 'Data\\Restaurants_London_England.tsv'
restaurants_list, borders = load_reviews_to_list(path)

tags_list, bags_of_restaurants = generate_inverted_index(restaurants_list)

x_list, y_list = create_grid(borders,50)
grid = add_restaurants_to_grid(x_list, y_list, restaurants_list)

In [25]:
def kwSpaSearchIF(r_query, t_query, tags_list, bags_of_restaurants, x_list, y_list, grid, restaurants_list):
    tags_results = kwSearchIF(t_query, tags_list, bags_of_restaurants)
    final_results = spaSearchGridAfterIF(r_query, tags_results, x_list, y_list, grid, restaurants_list)
    return final_results

def kwSpaSearchGrid(r_query, t_query, tags_list, bags_of_restaurants, x_list, y_list, grid, restaurants_list):
    grid_results = spaSearchGrid(r_query,x_list,y_list,grid,restaurants_list)
    final_results = kwSearchIFAfterGrid(t_query, tags_list, bags_of_restaurants, grid_results)
    return final_results

def kwSearchIFAfterGrid(text_list, tags_list, bags_list, grid_results):
    s_text_list = sorted(text_list)

    if_indexes = merge_join(s_text_list, tags_list)
    print(if_indexes)
    if if_indexes == -1:
        return []

    r_intersection = set([r for r in bags_list[if_indexes[0]] if r in grid_results])
    if len(if_indexes) > 1:
        for i in range(1, len(if_indexes)):
            r_intersection = r_intersection.intersection([r for r in bags_list[if_indexes[i]] if r in grid_results])

    return list(r_intersection)



def spaSearchGridAfterIF(r_query, tags_results, x_list, y_list, grid, restaurants_list):

    x_min_index = binary_search(x_list, r_query[0])
    x_max_index = binary_search(x_list, r_query[1])

    y_min_index = binary_search(y_list, r_query[2])
    y_max_index = binary_search(y_list, r_query[3])
        
    results = []
    for i in range(y_min_index,y_max_index + 1):
        for j in range(x_min_index,x_max_index + 1):
            if len(grid[i][j]) > 0:
                for r in grid[i][j]:
                    if r in tags_results and \
                        restaurants_list[r]['x'] >= r_query[0] and \
                        restaurants_list[r]['x'] <= r_query[1] and \
                        restaurants_list[r]['y'] >= r_query[2] and \
                        restaurants_list[r]['y'] <= r_query[3]:
                        results.append(r)

    return results

In [23]:
r_query = [51, 51.50, -0.5, 0]
t_query = ['british', 'bar']
kwspaif =  kwSpaSearchGrid(r_query, t_query, tags_list, bags_of_restaurants, x_list, y_list, grid, restaurants_list)

In [24]:
print(len(kwspaif))
for r in kwspaif:
    print(restaurants_list[r]['review_id'])
    print('location:', restaurants_list[r]['x'], ',', restaurants_list[r]['y'])
    print('tags:', restaurants_list[r]['tags'], '\n')

0
